In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_ascertain as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

phases = dr.CLIPS

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    # train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]


models = {
    # "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "RF": RandomForestClassifier(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=label_type, threshold=threshold, normalize=True)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.train_predict(models, x, y, by_subject=True, save_metrics=True, is_resample=False, get_importance=get_importance, drop_subject=True, test_size=0.2, folds=5)
    for model_name in acc_results:
        for i in range(len(out[model_name])):
            acc_results[model_name].append(out[model_name][i][0])
            reports[model_name].append(out[model_name][i][1])
        if get_importance:
            try:
                print("")
                # shap.plots.bar(out[model_name][0][2])
                for i in range(len(out[model_name])):
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][i][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
            except Exception as e:
                print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0.5:
        print(f"Model evaluation metrics for {model_name}:")
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
    print("")
print("\n")

[[53, 31, 52, 33, 26, 49, 43, 34, 24, 19, 22, 44], [25, 23, 40, 38, 20, 48, 17, 11, 51, 7, 54, 39], [1, 27, 5, 45, 10, 41, 15, 30, 21, 8, 12, 58], [37, 6, 14, 18, 29, 4, 3, 55, 16, 47, 50, 56], [32, 13, 2, 42, 46, 28, 35, 9, 36, 57]]
y_train | y_test:
{1: 1201, 0: 211} | {1: 321, 0: 51}
y_train | y_test:
{1: 1206, 0: 208} | {1: 316, 0: 54}
y_train | y_test:
{1: 1204, 0: 208} | {1: 318, 0: 54}
y_train | y_test:
{1: 1183, 0: 233} | {1: 339, 0: 29}
y_train | y_test:
{1: 1294, 0: 188} | {1: 228, 0: 74}
Fold #0
Model LogReg, Predictions: [0 1], [  1 371]
{'precision': 0.8652291105121294, 'recall': 1.0, 'f1': 0.9277456647398843, 'auc': 0.5098039215686274}
Fold #1
Model LogReg, Predictions: [0 1], [  1 369]
{'precision': 0.8536585365853658, 'recall': 0.9968354430379747, 'f1': 0.9197080291970804, 'auc': 0.49841772151898733}
Fold #2
Model LogReg, Predictions: [0 1], [  1 371]
{'precision': 0.8544474393530997, 'recall': 0.9968553459119497, 'f1': 0.9201741654571843, 'auc': 0.49842767295597484}
Fo